In [27]:
'''
Descripttion: numpy实现根据阈值给anchor分配gtbox
version: 1.0
Author: SongJ
Date: 2021-09-12 22:12:42
LastEditors: SongJ
LastEditTime: 2021-09-12 22:22:40
'''
import numpy as np

In [28]:
def box_iou(boxes1,boxes2):
    ''' input:
            boexs1.shape(boxes1的数量,xyxy)
            boexs2.shape(boxes2的数量,xyxy)
        return:
            (boxes1的数量,boxes2的数量)
    '''

    # 计算面积,areas.shape(boxes的数量,)
    box_area = lambda boxes : ((boxes[:,3]-boxes[:,1]) * (boxes[:,2]-boxes[:,0]))
    areas1, areas2 = box_area(boxes1),box_area(boxes2)

    #取相交部分的坐标值,利用广播机制,shape(boxes1的数量,boxes2的数量,2)
    interuppeerleft = np.maximum(boxes1[:,None,:2],boxes2[:,:2]) # 左上角坐标
    interlowerright = np.minimum(boxes1[:,None,2:],boxes2[:,2:]) # 右下角坐标

    #计算相交部分的面积,shape(boxes1的数量,boxes2的数量,2)
    inter = (interlowerright - interuppeerleft).clip(min=0) # 做差有负数表示不相交,直接设置为0
    # shape(boxes1的数量,boxes2的数量)
    interarea = inter[:,:,0] * inter[:,:,1] 

    # 计算并积,shape(boxes1的数量,boxes2的数量)
    unionarea = areas1[:,None] + areas2 - interarea
    return interarea/unionarea


In [29]:
anchors = np.random.randint(1,5,(5,4))
gt = np.random.randint(1,5,(3,4))


In [24]:
def assign_anchor_to_bbox(gt, anchors, device, iou_threshold=0.5):
    num_anchors, num_gt_boxes = anchors.shape[0], gt.shape[0]
    # 计算iou,jaccard.shape(num_anc,num_gt)
    jaccard = box_iou(anchors,gt)
    # 生成一个anc长度的填充-1的数组
    anchors_bbox_map = np.full((num_anchors,), -1, dtype=np.long)

    # 计算阈值,max_iou是每行最大的iou,indices是对应的列索引
    max_ious, indices = np.maximum(jaccard, dim=1)
    # 取出符合阈值的anc的行索引
    anc_i = np.nonzero(max_ious >= 0.5).reshape(-1)
    # 取出符合阈值的anc的列索引
    box_j = indices[max_ious > 0.5]
    # 将gt分配给符合条件的anc
    anchors_bbox_map[anc_i] = box_j
    col_discard = np.full((num_anchors,), -1)
    row_discard = np.full((num_gt_boxes,), -1)
    

/home/songj/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


In [20]:
jaccard

array([[-0., -0.,  0.],
       [ 0.,  0.,  0.],
       [ 0., nan,  0.],
       [ 0.,  0.,  0.],
       [-0., -0.,  0.]])